<a href="https://colab.research.google.com/github/ekerintaiwoa/MediaApp/blob/master/tbookchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import fitz  # PyMuPDF
import faiss
import gradio as gr
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
import numpy as np

In [6]:
# Load local LLaMA model (GGUF format)
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=8)
print("LLaMA model loaded successfully!")

ValueError: Model path does not exist: ./llama-3-8b-instruct.Q4_K_M.gguf

In [7]:
# Install dependencies
!pip install --quiet llama-cpp-python


In [9]:
# Download the model (about 3.5GB)
!wget -c https://huggingface.co/TheBloke/llama-2-7b-ggml-q4_0/resolve/main/llama-2-7b.ggml.q4_0.bin -O llama-2-7b.ggml.q4_0.bin


--2025-06-15 04:50:48--  https://huggingface.co/TheBloke/llama-2-7b-ggml-q4_0/resolve/main/llama-2-7b.ggml.q4_0.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.


In [10]:
from huggingface_hub import login

login()


In [17]:
from huggingface_hub import hf_hub_download

# Download a compatible Llama 2 GGML model
model_name = "llama-2-7b-chat.ggmlv3.q4_0.bin"
model_path = hf_hub_download(repo_id="TheBloke/Llama-2-7B-Chat-GGML", filename=model_name)
print(f"Model downloaded to: {model_path}")

Model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q4_0.bin


In [20]:
# prompt: load the model  with llama-cpp-python and test with sample question

# Define the path to your downloaded model
model_path = hf_hub_download(repo_id="TheBloke/Llama-2-7B-Chat-GGML", filename=model_name)

# Load the Llama model
llm = Llama(model_path=model_path, n_ctx=2048, n_threads=8)
print("LLaMA model loaded successfully!")

# Sample question to test the model
prompt = "What is the capital of France?"

# Generate a response
output = llm(prompt, max_tokens=150, stop=["Q:", "\n"], echo=True)

# Print the output
print(output["choices"][0]["text"])

llama_model_load: error loading model: llama_model_loader: failed to load model from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q4_0.bin

llama_model_load_from_file_impl: failed to load model


ValueError: Failed to load model from file: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGML/snapshots/76cd63c351ae389e1d4b91cab2cf470aab11864b/llama-2-7b-chat.ggmlv3.q4_0.bin

In [19]:
%pip install --upgrade --quiet llama-cpp-python

In [21]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Download a Llama 2 7B Chat GGUF model
gguf_model_name = "llama-2-7b-chat.Q4_K_M.gguf"
gguf_model_path = hf_hub_download(repo_id="TheBloke/Llama-2-7B-Chat-GGUF", filename=gguf_model_name)
print(f"GGUF model downloaded to: {gguf_model_path}")

# Load the GGUF model
llm_gguf = Llama(model_path=gguf_model_path, n_ctx=2048, n_threads=8)
print("GGUF LLaMA model loaded successfully!")

# You can now use llm_gguf for inference
# For example:
# prompt = "What is the capital of France?"
# output = llm_gguf(prompt, max_tokens=150, stop=["Q:", "\n"], echo=True)
# print(output["choices"][0]["text"])

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_co

GGUF model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf


llama_model_loader: - kv   8:              llama.attention.head_count_kv u32              = 32
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000001
llama_model_loader: - kv  10:                          general.file_type u32              = 15
llama_model_loader: - kv  11:                       tokenizer.ggml.model str              = llama
llama_model_loader: - kv  12:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv

GGUF LLaMA model loaded successfully!


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '11008', 'llama.attention.layer_norm_rms_epsilon': '0.000001', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '32', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '32', 'llama.attention.head_count_kv': '32', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '15'}
Using fallback chat format: llama-2


In [22]:
prompt = "What is the capital of France?"
output = llm_gguf(prompt, max_tokens=150, stop=["Q:", "\n"], echo=True)
print(output["choices"][0]["text"])

llama_perf_context_print:        load time =    5634.00 ms
llama_perf_context_print: prompt eval time =    5633.68 ms /     8 tokens (  704.21 ms per token,     1.42 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5637.95 ms /     9 tokens


What is the capital of France?


In [23]:
response = llm_gguf("Hello! Explain Newton's laws simply.", max_tokens=100)
print(response["choices"][0]["text"].strip())

Llama.generate: 1 prefix-match hit, remaining 10 prompt tokens to eval
llama_perf_context_print:        load time =    5634.00 ms
llama_perf_context_print: prompt eval time =    2594.43 ms /    10 tokens (  259.44 ms per token,     3.85 tokens per second)
llama_perf_context_print:        eval time =   39914.75 ms /    99 runs   (  403.18 ms per token,     2.48 tokens per second)
llama_perf_context_print:       total time =   42561.09 ms /   109 tokens


Newton's laws of motion are three laws that describe how objects move and interact with other objects in the universe. These laws were first discovered by Sir Isaac Newton in the late 1600s and are still used today to predict and understand the behavior of objects.
The first law, also known as the law of inertia, states that an object at rest will remain at rest, and an object in motion will continue to move in a straight line at a constant


In [24]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
# Step 1: Extract text from PDF
def extract_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [26]:
# Step 2: Chunk text into sections
def chunk_text(text, max_length=500):
    words = text.split()
    return [" ".join(words[i:i+max_length]) for i in range(0, len(words), max_length)]

In [27]:
# Step 3: Build vector index from chunks
def build_vector_store(chunks):
    embeddings = embedder.encode(chunks)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings))
    return index, embeddings, chunks

In [28]:
# Step 4: Ask question
def ask_question(question, index, chunks, embeddings):
    q_embed = embedder.encode([question])
    D, I = index.search(np.array(q_embed), k=3)
    context = "\n\n".join([chunks[i] for i in I[0]])

    prompt = f"""<|system|>You are a helpful assistant that answers questions based only on the context given.</s>
<|user|>Context: {context}
Question: {question}
Answer:</s><|assistant|>"""

    result = llm(prompt, max_tokens=300, stop=["</s>", "<|user|>"])
    return result["choices"][0]["text"].strip()

In [34]:
# try option
def ask_question(question, index, chunks, embeddings):
    try:
        q_embed = embedder.encode([question])
        q_embed = np.array(q_embed).reshape(1, -1)
    except Exception as e:
        return f"Embedder error: {e}"

    try:
        D, I = index.search(q_embed, k=3)
        context = "\n\n".join([chunks[i] for i in I[0] if i >= 0 and i < len(chunks)])
    except Exception as e:
        return f"Index search error: {e}"

    prompt = f"""<|system|>You are a helpful assistant that answers questions based only on the context given.</s>
<|user|>Context: {context}
Question: {question}
Answer:</s><|assistant|>"""

    try:
        result = llm_gguf(prompt, max_tokens=300, stop=["</s>", "<|user|>"])
        return result["choices"][0]["text"].strip()
    except Exception as e:
        return f"LLaMA model error: {e}"


In [38]:
# Step 5: Gradio Interface
def process_pdf_and_ask(pdf_file, question):
    if pdf_file is None or question.strip() == "":
        return "Please upload a book and ask a question."

    text = extract_text(pdf_file.name)
    chunks = chunk_text(text)
    index, embeddings, stored_chunks = build_vector_store(chunks)
    answer = ask_question(question, index, stored_chunks, embeddings)
    return answer

In [39]:
# Gradio UI
interface = gr.Interface(
    fn=process_pdf_and_ask,
    inputs=[
        gr.File(label="Upload PDF Book"),
        gr.Textbox(label="Ask a Question About the Book")
    ],
    outputs="text",
    title="📚 Offline Book Q&A Chatbot",
    description="Upload a book (PDF) and ask any question about it. Powered by a local LLaMA model."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://770d94b340db1b64f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [37]:
# prompt: got this error from the chat bot: LLaMA model error: Requested tokens (2074) exceed context window of 2048

def ask_question(question, index, chunks, embeddings):
    try:
        q_embed = embedder.encode([question])
        q_embed = np.array(q_embed).reshape(1, -1)
    except Exception as e:
        return f"Embedder error: {e}"

    try:
        D, I = index.search(q_embed, k=3)
        context = "\n\n".join([chunks[i] for i in I[0] if i >= 0 and i < len(chunks)])
    except Exception as e:
        return f"Index search error: {e}"

    # Limit the context length to avoid exceeding the context window
    # A common approach is to truncate or summarize the context
    # For simplicity here, we'll just truncate if it's too long
    max_context_length = 1500 # Adjust this value based on your model's actual capability after prompt template
    if len(context) > max_context_length:
      context = context[:max_context_length] # Simple truncation

    prompt = f"""<|system|>You are a helpful assistant that answers questions based only on the context given.</s>
<|user|>Context: {context}
Question: {question}
Answer:</s><|assistant|>"""

    try:
        # Adjust max_tokens for the response if needed, but the primary issue is the input context
        result = llm_gguf(prompt, max_tokens=300, stop=["</s>", "<|user|>"])
        return result["choices"][0]["text"].strip()
    except Exception as e:
        return f"LLaMA model error: {e}"

